# Exploratory Data Analysis of Freelance Profiles with relation to Hourly Rate

In [1]:
# Notes to self:

# The clustering on skills didn't work because there isn't enough skills per user.
# Almost all users have about 5 skills, but there isn't enough overlap between them.
# Futhermore, there is just about no signal relating the features to hourly rate.
# Need to pivot and think of some alternative way to use this data.
# Also, I need to figure out why everyone has a "1" for their invoices paid.

# Do a regression tree model
# Deriving other features from the text
# Sentiment: Passive vs active voice?
# do the skills appear in the text 
# Try a targeted search and get numerics out
    # remove +'s' or try regex 
# Use text to look for those skills and maybe try to find words around it?
# Skills in text vs not in the skill list 
# Naively do some kind of embedding (word2vec)
    # Toss in their description and it creates a vector representation 
    # And then try looking at clusters in that space
    # Specify the length of the vector 

### Importing and Merging to get EDA Dataset

In [2]:
# Packages for PostgreSQL Import and Export
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy import create_engine
import psycopg2

# Packages for EDA
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
from nltk.corpus import stopwords
import re 

# Packages for K-Modes Cluster
import numpy as np
from kmodes.kmodes import KModes

In [3]:
# Ideally I'll move this into the project config.py file
# Otherwise for now I have to just manually assign . . .

dbname = "freelance_db"
username = "Metaverse"
pswd = "Arcifice91"

# Connect to Data
con = None
con = psycopg2.connect(database=dbname, user=username,
                       host='localhost', password=pswd)

# Checking shapes of tables
sql_query = """SELECT * from user_details_table;"""
dtls_table = pd.read_sql_query(sql_query, con)

sql_query = """SELECT * from freelance_table;"""
fl_table = pd.read_sql_query(sql_query, con)

In [4]:
# Merging
# Modifying profile_url in the details table so I can merge it with the fl_table
dtls_table['profile_url'] = "https://www.guru.com"+dtls_table['profile_url']

# Now I'm merging them together to make dt
dt = pd.merge(fl_table, dtls_table, on='profile_url')
dt.shape
dt = dt.drop(columns=(['index_x']))
dt = dt.drop_duplicates(subset='profile_url')

In [5]:
# Merging in data to crosswalk states with region labels
state_region_crswlk = pd.read_csv(
    '~/Desktop/Insight/projects/myrate/data/raw/census-regions/us census bureau regions and divisions.csv')
state_region_crswlk.head()

dt = pd.merge(dt, state_region_crswlk,
              how='left', left_on='state', right_on='State')

# Fixing region value for individuals living in Puerto Rico
dt.loc[dt['state'] == "Puerto Rico", 'Region'] = "Other"

# Exploratory Data Analysis - Outcome: Hourly Rate
### Beginning with investigating the geographic labels 

In [375]:
# What is the overall mean?
print("Mean hourly rate ($):", round(dt['hourly_rate'].mean(), 2))

Mean hourly rate ($): 31.94


In [ ]:
# What is the mean by region?
dt.groupby(['Region']).hourly_rate.mean()

# There is some variation, but it is minimal

In [ ]:
# What is the mean across states?
dt.groupby(['state']).hourly_rate.mean().sort_values().describe()

# Pretty decent spread. This may be the most powerful signal yet.
# If you print out hte full table it is clear that there is variation

In [376]:
# What is the variation within states?
state_lvl_hourly_rate_descriptives = {}
state_lvl_hourly_rate_descriptives['min'] = dt.groupby(
    ['state']).hourly_rate.min()
state_lvl_hourly_rate_descriptives['mean'] = dt.groupby(
    ['state']).hourly_rate.mean()
state_lvl_hourly_rate_descriptives['max'] = dt.groupby(
    ['state']).hourly_rate.max()
state_lvl_hourly_rate_descriptives['var'] = dt.groupby(
    ['state']).hourly_rate.var()
dt_state_hr = pd.DataFrame(state_lvl_hourly_rate_descriptives)

# A decent amount of variation within the states

In [ ]:
# I tried to break it down further to the city level
# But the city data is really messy and not worth the time sink
# To try and get it set up. The idea would be to merge on
# rural vs urban and see if it can help me filter out a signal.

### Investigating the relationship between a users "rating" and the outcome

In [6]:
# Cleaing it up real quick
dt.rating = dt.rating.str.replace('%', '')
dt.rating = dt.rating.str.replace('NA', '')
dt.rating = pd.to_numeric(dt.rating)

In [ ]:
# Starting with Missing Values
print("Number of missing values: ", dt.rating.isna().sum())

In [ ]:
# What does the spread look like for the rest?
dt.rating.describe()

# Basically no variation. Okay, let's make a binary variable (has_rating)

In [7]:
# Creating "has_rating"
dt['has_rating'] = None
dt.loc[dt['rating'].isna(), 'has_rating'] = 0
dt.loc[dt['rating'] > 0, 'has_rating'] = 1

# Comparing outcome means by has_rating
dt.groupby(['has_rating']).hourly_rate.mean()

# Small difference, nothing substantial

has_rating
0    34.529412
1    31.743139
Name: hourly_rate, dtype: float64

### Exploring the duration of membership and hourly rate

1. Convert member start date to a date object
2. Calculate years / months they've been a member (to today)

In [8]:
def date_convert(member_since):
    try:
        tmp = datetime.strptime(member_since, '%b, %Y')
        #tmp = tmp.strftime('%Y-%m')
    except:
        tmp = 'NaN'
    return tmp

In [9]:
def years_active(date):
    cur_year = datetime.now().year
    try:
        yrs_active = cur_year - date.year
    except:
        yrs_active = 'NaN'

    return yrs_active

In [10]:
def months_active(date):
    cur_year = datetime.now().year
    cur_month = datetime.now().month

    try:
        mnths_active = (cur_year - date.year)*12 + (cur_month - date.month)
    except:
        mnths_active = 'NaN'

    return mnths_active

In [11]:
# Converting member_since to date
dt.member_since = dt.member_since.str.strip()
dt['start_date'] = dt.member_since.apply(date_convert)
dt['years_active'] = dt.start_date.apply(years_active)
dt['months_active'] = dt.start_date.apply(months_active)

# Therea are 31 NAs. Just going to ignore them for now.

In [12]:
# What I want to understand is:
# How does the hourly rate (average) change across years active?
dt.groupby('years_active').hourly_rate.mean()

# Nothing much. It ramps up if years active > 17. Building that binary.

years_active
0.0     30.869565
1.0     32.279570
2.0     31.409091
3.0     31.804598
4.0     30.594203
5.0     30.295775
6.0     29.396552
7.0     23.527778
8.0     31.758621
9.0     24.173913
10.0    28.482759
11.0    28.192308
12.0    34.178571
13.0    32.521739
14.0    37.481481
15.0    38.652174
16.0    32.541667
17.0    51.190476
18.0    39.384615
19.0    42.533333
20.0    55.000000
Name: hourly_rate, dtype: float64

In [13]:
dt['active_17up'] = (dt['years_active'] >= 17) * 1

In [ ]:
# Okay let's try looking at months active. Maybe on a finer scale there is some difference?
x = dt.groupby('months_active').hourly_rate.mean().index
y = dt.groupby('months_active').hourly_rate.mean()
ax = sns.scatterplot(x=x, y=y,
                     color='black')
ax.set(xlabel='months_active', ylabel='Average hourly_rate')
plt.show()

# No real pattern emerges. Unfortunately not useful.

### Exploring hourly rate and how it relates to the skills

1. Obtain the dataset that is long by skills and shrink it down to just the skills. I'll work with that for this.
2. Create some quick features such as: # of skills and top skill. These are user level so I can attach them back to dt to have everything in one place.

Note: the skills are standardized on guru which makes this more straightforward

In [14]:
sql_query = """SELECT profile_url, skills_list from freelance_table;"""
skills_table = pd.read_sql_query(sql_query, con)
skills_table_dummies = pd.get_dummies(skills_table, columns=[
                                      'skills_list'], prefix='').groupby(['profile_url']).sum()

In [15]:
# Number of skills
num_skills = skills_table.groupby('profile_url').count()
num_skills = num_skills.rename(columns={"skills_list": "num_skills"})
num_skills = num_skills.reset_index()

In [16]:
# Merging num skills into dt
dt = pd.merge(dt, num_skills, on="profile_url")

In [17]:
dt.groupby('num_skills').hourly_rate.mean()

num_skills
1    25.620690
2    42.600000
3    27.750000
4    24.381818
5    32.415854
Name: hourly_rate, dtype: float64

In [18]:
# Seems that for those with less than five skills listed
# they tend to have a lower hourly rate.
dt['less_five_skills'] = (dt['num_skills'] < 5) * 1

In [19]:
# Moving on to something else. Let's look at the top skills (first skill listed)
first_skill = skills_table[skills_table.groupby(
    'profile_url')['skills_list'].cumcount() == 0]

first_skill = first_skill.rename(
    columns={"profile_url": "profile_url", "skills_list": "first_skill"})

dt = pd.merge(dt, first_skill, on="profile_url")

In [20]:
# Exploring First Skills
print("Number of unique skills: ", dt.first_skill.nunique())
dt.groupby('first_skill').first_skill.count().describe()

# There is some repition but not much. Maybe I can get something off the extreme cases.

Number of unique skills:  350


count    350.000000
mean       2.797143
std        4.245543
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       32.000000
Name: first_skill, dtype: float64

In [21]:
# Exploring extreme first skill frequencies
skill_freq = {}
skill_freq['skill_frequency'] = dt.groupby(
    'first_skill').first_skill.count().sort_values(ascending=False)
skill_freq['skills_list'] = dt.groupby(
    'first_skill').first_skill.count().sort_values(ascending=False).index
skill_freq = pd.DataFrame(skill_freq)
skill_freq = skill_freq.reset_index()
skill_freq = skill_freq.drop(['first_skill'], axis=1)

dt = pd.merge(dt, skill_freq, on="skills_list")

In [ ]:
sns.scatterplot(x='skill_frequency', y='hourly_rate', data=dt)

# Nope, not much of anything here.

In [ ]:
# Just have to try to keep thinking of ways I could use this information.
# Ideally, I would be able to go group them by category and calculate hourly rate on that
# There may also be an interaction effect occuring between the category and something like
# years of experience.

### Starting the process of using the "bio" data

1. The easiest variable to create is the len(bio)
2. I'll have to try and expand it more using some other techniques

In [22]:
def clean_bio(bio):
    try:
        cleaned_bio = ''.join(s for s in bio if ord(s) > 31 and ord(s) < 126)
    except:
        cleaned_bio = "NaN"
    return cleaned_bio

In [23]:
def len_bio(bio):
    if bio == "NaN":
        return 0
    else:
        return len(bio)

In [24]:
# Average word length in bio
def avg_word_ln(bio):
    try:
        words = bio.split()
        res = (sum(len(word) for word in words)/len(words))
    except:
        res = 0
    return res

In [25]:
# Number of stop words
def num_stopwords(bio):
    stop = stopwords.words('english')

    try:
        res = len([x for x in bio.split() if x in stop])
    except:
        res = -1

    return res

In [26]:
dt['bio_clean'] = dt.bio.apply(clean_bio)

In [27]:
dt['bio_length'] = dt.bio_clean.apply(len_bio)

In [3]:
len("Hi how are you doing today".split(" "))

6

In [28]:
# Number of separate words in bio
dt['bio_word_count'] = dt['bio_clean'].apply(
    lambda x: len(str(x).split(" ")))

In [29]:
dt['avg_word_length'] = dt['bio_clean'].apply(avg_word_ln)

In [30]:
dt['num_stop'] = dt['bio_clean'].apply(num_stopwords)

In [31]:
# Preprocessing before pulling more features

# Remvoing stop words
stop = stopwords.words('english')
dt['bio_processed'] = dt['bio_clean'].apply(
    lambda x: " ".join(x for x in x.split() if x not in stop))

# Removing punctuation
dt['bio_processed'] = dt['bio_processed'].str.replace(
    '[^\w\s]', '')

# Lower Case
dt['bio_processed'] = dt['bio_processed'].apply(
    lambda x: " ".join(x.lower() for x in x.split()))

In [ ]:
# Exploring how these bio features related to hourly rate

# Starting with the length of a users bio
sns.scatterplot(x='bio_length', y='hourly_rate', data=dt)

# Nothing.

In [ ]:
# How about how the word count related to hourly rate?
sns.scatterplot(x='bio_word_count', y='hourly_rate', data=dt)

# Nope, again nothing of interest here.

In [ ]:
# Okay, what about how word length
sns.scatterplot(x='hourly_rate', y='avg_word_length', data=dt)

# Hmm, no. But there are some very unusual outliers.

In [ ]:
# Lastly, the number of stop words
sns.scatterplot(x='num_stop', y='hourly_rate', data=dt)

In [ ]:
# Whole lot of nothing came out of the bio features.

### Comparing number of employers and hourly rate

Nevermind. There is no variance.

### Some final cleaning

In [32]:
# Making State dummy variables
state_dummies = pd.get_dummies(dt['state'])
dt = pd.concat([dt, state_dummies], axis=1)

In [33]:
# Earnings per Month
dt['earnings_ever'] = dt['earnings_ever'].str.replace('$', '')
dt['earnings_ever'] = dt['earnings_ever'].str.replace(',', '')
dt['earnings_ever'] = pd.to_numeric(dt['earnings_ever'])
dt['earnings_pr_month'] = dt['earnings_ever'] / \
    dt['months_active']

# Exploratory Data Analysis of Skills and Labels

### Goal: Merge scraped skills list and categories into data based on user reported skills

### Importing Data

In [34]:
sql_query = """SELECT * from skills_categories_table;"""
sk_table = pd.read_sql_query(sql_query, con)
sk_table = sk_table.drop(['index'],axis = 1)

In [35]:
# Extracting Numbers
sk_table['Skill_Frequency'] = pd.to_numeric(sk_table.Skills.str.extract(
    r'\((\d*?)\)').iloc[:, 0])

In [36]:
# Cleaning Columns of frequency labels
sk_table['Skills_Clean'] = sk_table.Skills.str.replace(
    r'\((\d*?)\)','').str.strip().str.lower()
sk_table['Category'] = sk_table.Category.str.replace(
    r'\((.*?)\)','').str.strip().str.lower()
sk_table['Subcategory'] = sk_table.Subcategory.str.replace(
    r'\((\d*?)\)','').str.strip().str.lower()

In [37]:
# Pulling in the user skills table
sql_query = """SELECT profile_url, skills_list from freelance_table;"""
skills_table = pd.read_sql_query(sql_query, con)

In [38]:
skills_table['skills_list'] = skills_table['skills_list'].str.lower().str.strip()

In [39]:
tmp = pd.merge(skills_table,sk_table,
               left_on='skills_list',right_on='Skills_Clean', how="left",
               indicator=True)

In [40]:
# Now I want to try and see how many people match
# If they matched on at least one thing I can then expand that onto the rest of their data
tmp['matched'] = (tmp['_merge'] == "both") * 1
number_of_matches_by_user = tmp.groupby(['profile_url']).matched.sum()

In [39]:
(number_of_matches_by_user == 0).sum()

# This is great, a good chunk of people matched.
# Now I can guess what field they were in (for the ones who matched)
# and then label the people similar to them

268

In [41]:
tmp['num_matched'] = tmp.groupby(['profile_url']).matched.transform('sum')

In [42]:
# So for people who matched, I want to determine which category most common for them
test = tmp[tmp['num_matched'] > 0][['profile_url','Category']]

In [43]:
# Reshaping wide
# test['counter'] = test.groupby(['profile_url']).cumcount()+1
test['counter'] = 1
skill_category_matrix = test.pivot_table(index = 'profile_url', columns = 'Category', 
                     values = 'counter', aggfunc = 'sum', 
                     fill_value = 0)
skill_category_matrix.head()

Category,administrative & secretarial,business & finance,design & art,education & training,engineering & architecture,legal,programming & development,sales & marketing,writing & translation
profile_url,,,,,,,,,
https://www.guru.com/freelancers/35-year-editor-has-your-back,0,0,0,0,0,0,0,0,5
https://www.guru.com/freelancers/86-keys-communications,0,0,0,0,0,0,0,0,2
https://www.guru.com/freelancers/a-stringham,2,0,0,0,0,0,0,0,1
https://www.guru.com/freelancers/a-wordsmith,0,0,0,0,0,0,0,0,2
https://www.guru.com/freelancers/aa-consultants,0,1,0,0,0,0,2,0,0


# Quick Update: Working with Skill Categories

1. I now have a user-level matrix with the skill categories along the top
2. I also have a user-level matrix with skill categories along the top but with missing data.
3. I want to impute that matrix. How?
4. The simplest way would be to create the **very** wide skill matrix and do lookups for similarity.
    - For example, take skills vector (long, 0/1s) of user A who I want to impute his categories. Find the most similar vector among the users with data. Impute from there.

In [44]:
# First resetting index on skill_category_matrix
skill_category_matrix = skill_category_matrix.reset_index()
skill_category_matrix['has_categories'] = 1

In [45]:
# And creating skills dummy matrix
sql_query = """SELECT profile_url, skills_list from freelance_table;"""
skills_table = pd.read_sql_query(sql_query, con)
skills_table_dummies = pd.get_dummies(skills_table, columns=[
                                      'skills_list'], prefix='').groupby(['profile_url']).sum()
skills_table_dummies = skills_table_dummies.reset_index()

In [46]:
# And merging
skills_table_dummies = pd.merge(skills_table_dummies, 
                                skill_category_matrix.loc[:,['profile_url','has_categories']], 
                                left_on = 'profile_url',
                                right_on = 'profile_url',
                                how = 'left')

In [47]:
# Filling in the NA has_categories so it's a useful indicator
skills_table_dummies['has_categories'] = skills_table_dummies['has_categories'].fillna(0)

In [48]:
has_cat = skills_table_dummies[skills_table_dummies['has_categories'] == 1]
no_cat = skills_table_dummies[skills_table_dummies['has_categories'] == 0]

In [49]:
def raw_distance():
    sqrdist = np.sum(np.sqrt(np.square(user_no_cat-user_has_cat)))
    return sqrdist

In [50]:
# Creating one long list of all distances
dist = []
for k in range(0,no_cat.shape[0]):
    if k%50 == 0 :
        print(k) 
    for j in range(0,has_cat.shape[0]):
        user_no_cat = np.array(list(no_cat.iloc[k,1:]))
        user_has_cat = np.array(list(has_cat.iloc[j,1:]))
        dist.append(raw_distance())

    

0
50
100
150
200
250


In [51]:
# Putting that list into a numpy matrix of the appropriate shape
res = np.array(dist).reshape(no_cat.shape[0],has_cat.shape[0])

In [52]:
# Finding where the FIRST minimum is for each user
# Come back later and clean this up to return multiple minimums
# Making my life easy for now . . .
min_indices = np.argmin(res, axis = 1) 

In [53]:
min_indices[1]

101

In [54]:
# Now I want to take the minimum index and use it on the has_cat table
# to look up the person with whom they have similar relationship.
# I can then extract this users row and slap it into a new table (impute_cat)
cols = list(skill_category_matrix.columns)[0:10]
impute_cat = pd.DataFrame(columns=cols)

impute_cat['profile_url'] = no_cat['profile_url']

for j in range(0,no_cat.shape[0]):
    impute_cat.iloc[j,1:] = skill_category_matrix.iloc[min_indices[j],1:10]

In [55]:
# Append to skill_category_matrix
full_cat = skill_category_matrix.append(impute_cat)
full_cat = full_cat.drop(['has_categories'], axis = 1)
full_cat = full_cat.fillna(0)
# Done!

In [56]:
# Saving this out so I don't have to run the calculation again . . .
engine = create_engine('postgresql://%s:%s@localhost/%s' %
                       (username, pswd, dbname))
full_cat.to_sql("full_cat_dt", engine, if_exists='replace')

### Skill Categories Update #2

1. I imputed the skill categories using the L2 norm to find similar users
2. Now I have a user level matrix with the skill category and the number of skills they have in it.
3. However, the number is a little misleading because sometimes they just didn't match on their other listed skills.
4. Therefore, I now have the question of:
    - Do I create dummies for whether or not they have a certain category, at all?
    - Or do I pick the one they have the largest value in?
    - I think I may just do dummies for now. This means I just need to make any value > 0 = 1
5. Anyways, the next step is to explore the mean hourly_rate by skill categories.

In [57]:
# Merging it into the data
dt = pd.merge(dt, full_cat, on="profile_url")

In [58]:
# Making Binary Dummies
cols = list(dt.columns[-9:])
for j in range(0,len(cols)):
    dt.loc[dt[cols[j]] > 0, cols[j]] = 1

### Exploring the relationship between skill categories and hourly rate

In [ ]:
for j in range(0,len(cols)):
    print(dt.groupby(cols[j]).hourly_rate.mean())
    
# For Engineering, Legal, programming, there is a marked difference.

### Saving out Modified Dataset

1. First printing the columns I have for reference.
2. Saving it to a new table in the postgreSQL database

In [60]:
dt.head()

,profile_url,city,state,country,rating,earnings,hourly_rate,skills_list,user_description,index_y,...,earnings_pr_month,administrative & secretarial,business & finance,design & art,education & training,engineering & architecture,legal,programming & development,sales & marketing,writing & translation
0,https://www.guru.com/freelancers/scopic,Rutland,Massachusetts,United States,100.0,"$1,338,285/year",24,Angular,Scopic Software offers high-quality and afford...,0,...,27750.980296,0,0,0,0,0,0,0,1,0
1,https://www.guru.com/freelancers/crazy-program...,Hammond,Louisiana,United States,100.0,"$110,080/year",60,Angular,I am a full stack web developer with much expe...,5,...,12018.885714,0,0,0,0,0,0,1,0,0
2,https://www.guru.com/freelancers/andrii-tsyniuk,San Antonio,Texas,United States,100.0,"$45,320/year",50,Angular,Javascript developer with 8 years of experienc...,15,...,6474.285714,0,0,0,0,0,0,0,1,0
3,https://www.guru.com/freelancers/tyler-william...,Fort Collins,Colorado,United States,NaN,"$8,000/year",55,Angular,I am a full-stack web engineer with over 7+ ye...,19,...,579.875000,0,0,0,0,0,0,1,0,0
4,https://www.guru.com/freelancers/jonathan-ryan...,Hamilton Township,New Jersey,United States,100.0,"$4,400/year",70,Angular,I am available to write custom websites and ap...,18,...,162.500000,0,0,0,0,0,0,1,0,0


In [59]:

dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 979 entries, 0 to 978
Data columns (total 96 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   profile_url                   979 non-null    object        
 1   city                          979 non-null    object        
 2   state                         979 non-null    object        
 3   country                       979 non-null    object        
 4   rating                        911 non-null    float64       
 5   earnings                      979 non-null    object        
 6   hourly_rate                   979 non-null    int64         
 7   skills_list                   979 non-null    object        
 8   user_description              979 non-null    object        
 9   index_y                       979 non-null    int64         
 10  member_since                  979 non-null    object        
 11  earnings_pst_yr               94

In [60]:
col_locs = [4,6,21,24,26,31,32,33,34]
x = list(range(36,86))
y = list(range(87,96))
col_locs.extend(x)
col_locs.extend(y)
print(col_locs)

[4, 6, 21, 24, 26, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95]


In [62]:
# Cleaning to just variables that will be used in the model
analysis_dt = dt.iloc[:,col_locs]
analysis_dt = analysis_dt.dropna()  # Removing people with NA
analysis_dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 911 entries, 0 to 965
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   rating                        911 non-null    float64
 1   hourly_rate                   911 non-null    int64  
 2   has_rating                    911 non-null    object 
 3   months_active                 911 non-null    float64
 4   num_skills                    911 non-null    int64  
 5   bio_length                    911 non-null    int64  
 6   bio_word_count                911 non-null    int64  
 7   avg_word_length               911 non-null    float64
 8   num_stop                      911 non-null    int64  
 9   Alabama                       911 non-null    uint8  
 10  Arizona                       911 non-null    uint8  
 11  Arkansas                      911 non-null    uint8  
 12  California                    911 non-null    uint8  
 13  Color

In [63]:
# Connect to the database and save data to it
engine = create_engine('postgresql://%s:%s@localhost/%s' %
                       (username, pswd, dbname))
analysis_dt.to_sql("analysis_table", engine, if_exists='replace')

### OLD: Working on clustering the skills

1. This didn't work. I think the reason is because I don't have enough skills for each user. Therefore, it can't find enough similarity between them. Maybe I could go back and jump into each users url and scrape their entire skills list? Time consuming.

In [ ]:
# Found K-Modes on stackoverflow
# Following the package documentation
# Ref: https://pypi.org/project/kmodes/
# Ref: https://stackoverflow.com/questions/42639824/python-k-modes-explanation
# Ref: https://www.kaggle.com/ashydv/bank-customer-clustering-k-modes-clustering

In [ ]:
# Cleaning column names from the dummies database
skills_table_dummies.columns = skills_table_dummies.columns.str.strip().str.lower().str.replace(
    ' ', '_').str.replace('(', '').str.replace(')', '').str.replace('_', '')

In [ ]:
skills_table_dummies.head()

In [ ]:
# Modeling with K Modes
cost = []
for num_clusters in list(range(20, 30)):
    kmode = KModes(n_clusters=num_clusters, init="Huang", verbose=0)
    kmode.fit_predict(skills_table_dummies)
    cost.append(kmode.cost_)
    print("Finished Cluster: " + str(num_clusters))

In [ ]:
y = np.array([i for i in range(20, 30, 1)])
plt.plot(y, cost)

In [ ]:
kmode = KModes(n_clusters=28, init='Huang', verbose=0)
clusters = kmode.fit_predict(skills_table_dummies)

kmodes = kmode.cluster_centroids_
shape = kmodes.shape

for i in range(shape[0]):
    if sum(kmodes[i, :]) == 0:
        print("\ncluster " + str(i) + ": ")
        print("no-skills cluster")
    else:
        print("\ncluster " + str(i) + ": ")
        cent = kmodes[i, :]
        for j in skills_table_dummies.columns[np.nonzero(cent)]:
            print(j)

In [ ]:
clust_assigned = km.predict(skills_table_dummies)

In [ ]:
unique, counts = np.unique(clust_assigned, return_counts=True)
dict(zip(unique, counts))

In [ ]:
user_cluster_crosswalk = pd.DataFrame(
    skills_table['profile_url'].unique(), columns=["profile_url"]).sort_values(by="profile_url")
user_cluster_crosswalk['cluster'] = clust_assigned
user_cluster_crosswalk[user_cluster_crosswalk['cluster'] == 0].head()

In [ ]:
# Merging into the skills table to investigate those with "no skills"
tmp = pd.merge(skills_table, user_cluster_crosswalk, on="profile_url")
tmp[tmp['cluster'] == 0]

In [ ]:
# It seems to not be working great.
# Why is it putting so many people into the no skills cluster?

# EDA for the Probability of Getting a Job (0/1)

### Outcome: Probability of getting a job  . . . ever?

1. Need to clean all of this up

In [ ]:
# What about the (obvious) relationship between memebershp time and # of jobs completed
# How strong is the relationship?
# First have to clean the invoices paid variable
dt.invoices_paid = dt.invoices_paid.str.replace(',', '')
dt.invoices_paid = pd.to_numeric(dt.invoices_paid)
dt.invoices_paid.describe()

In [ ]:
# Who is NA? I have a whole bunch of 1s but then some NAs?
# The question for a poisson model is: did they just have no invoices OR
# were they never going to have any invoices.
# Looked at some of them and I'll treat them as zeros. So there are really that many ones?!?
dt[dt.invoices_paid.isna()].head()

In [ ]:
# Assigning zero to the NAs. This is important!!
dt.loc[dt.invoices_paid.isna(), 'invoices_paid'] = 0

In [ ]:
sns.distplot(dt.invoices_paid)

In [ ]:
# Wow, very heavy on the zeros. Let's look closer.
dt.invoices_paid.describe()

In [ ]:
# Ah, so it is truncating at one.
# This is going to be problematic for those who are actually at a value of one.
# I can either rerun the scrape and create something to make the val = 0
# or I can just move forward. . .

# For now I'm going to keep moving forward and treat 1 as 0.
# Note, it is more likely than not that they are = 0

In [ ]:
# Checking dist vs months active (no 1s)
sns.scatterplot(x='months_active', y='invoices_paid',
                data=dt[dt.invoices_paid > 1])

In [ ]:
# A few massive outliers. Going to rough chop it down and replot
invoice_gtr_one = dt.invoices_paid > 1
invoice_ls_2000 = dt.invoices_paid < 2000
sns.scatterplot(x='months_active', y='invoices_paid',
                data=dt[invoice_gtr_one & invoice_ls_2000])

In [ ]:
# hmm, there may be a slight signal but you would expect a stronger trend

In [ ]:
# Checking dist vs years active (no 1s)
sns.scatterplot(x='years_active', y='invoices_paid',
                data=dt[invoice_gtr_one & invoice_ls_2000])
# Slight postive non-linear trend bu there is a HUGE outlier

### Invoices Paid - Notes

It has a weak-ish relationship with time. There are a lot of really heavy users of the platform that dominate the market. Let's zoom into the first year and see what it looks like.

In [ ]:
# Looking at invoices pad in the first few months (among those only active for a shorter period of time)
sns.scatterplot(x='months_active',
                y='invoices_paid',
                data=dt[dt['years_active'] < 2])

In [ ]:
# Calculating invoices / time active
dt['invoices_per_month'] = dt['invoices_paid'] / \
    dt['months_active']
sns.distplot(dt.invoices_per_month)
print(dt.invoices_per_month.describe())

In [ ]:
# Loaded on zero with a long tail. Continues to sell that there are some serious power users.
# Average invoice per month is equal to one, but I left in the 1s that could be zeros. . .
# Real average is likely near zero.

# The question now is: Does this mean differ by any characteristic that I observe in the data?
# For people with a different skill set (Andriod) do they have a higher mean invoice/month rate?
# For people in different regions do they have a higher mean invoice/month rate?
# For people with lower hourly rates do they have a higher mean invoice/month rate??

# The next important step is to figure out how to use the skills data.
# It would be great to try and create clusters of the skills.